In [1]:
import pandas as pd

In [36]:
shark_class = '/m/0by6g'
ray_class = '/m/0m53l'
dolphin_class = '/m/02hj4'
split = 'train'

In [10]:
human_verified_df = pd.read_csv(f'../raw/human_{split}.csv')
bbox_df = pd.read_csv(f'../raw/{split}-annotations-bbox.csv')

In [37]:
class_of_interest = dolphin_class

In [38]:
labeled_images = human_verified_df.loc[human_verified_df['LabelName'] == class_of_interest]

In [39]:
labeled_images.count()

ImageID       2186
Source        2186
LabelName     2186
Confidence    2186
dtype: int64

In [41]:
# bbox_filtered = bbox_df.loc[bbox_df['ImageID'].isin(shark_images['ImageID']) & bbox_df['LabelName'].isin(shark_images['LabelName'])]

# Filter the bbox_df dataframe based on the ImageID column in shark_images and LabelName column in shark_images
# bbox_filtered = bbox_df.loc[bbox_df['ImageID'].isin(labeled_images['ImageID']) & bbox_df['LabelName'].isin([ray_class])]
# shark_filtered = bbox_df.loc[bbox_df['ImageID'].isin(labeled_images['ImageID']) & bbox_df['LabelName'].isin([shark_class])]
bbox_filtered = bbox_df.loc[bbox_df['LabelName'].isin([class_of_interest])]
shark_filtered = bbox_df.loc[bbox_df['ImageID'].isin(bbox_filtered['ImageID']) & bbox_df['LabelName'].isin([shark_class])]

print(f'Human-checked bboxes {bbox_filtered.count()}')
print(f'Human-checked bboxes with sharks {shark_filtered.count()}')

# Remove from bbox_filtered the images that have sharks
bbox_filtered = bbox_filtered.loc[~bbox_filtered['ImageID'].isin(shark_filtered['ImageID'])]
print(f'Human-checked bboxes without sharks {bbox_filtered.count()}')

Human-checked bboxes ImageID        1532
Source         1532
LabelName      1532
Confidence     1532
XMin           1532
XMax           1532
YMin           1532
YMax           1532
IsOccluded     1532
IsTruncated    1532
IsGroupOf      1532
IsDepiction    1532
IsInside       1532
XClick1X       1532
XClick2X       1532
XClick3X       1532
XClick4X       1532
XClick1Y       1532
XClick2Y       1532
XClick3Y       1532
XClick4Y       1532
dtype: int64
Human-checked bboxes with sharks ImageID        24
Source         24
LabelName      24
Confidence     24
XMin           24
XMax           24
YMin           24
YMax           24
IsOccluded     24
IsTruncated    24
IsGroupOf      24
IsDepiction    24
IsInside       24
XClick1X       24
XClick2X       24
XClick3X       24
XClick4X       24
XClick1Y       24
XClick2Y       24
XClick3Y       24
XClick4Y       24
dtype: int64
Human-checked bboxes without sharks ImageID        1500
Source         1500
LabelName      1500
Confidence     1500
XMin  

In [ ]:
# Create a new dataframe with ImageID, XMin, XMax, YMin, and YMax columns
new_df = bbox_filtered.drop([c for c in bbox_filtered.columns if c not in ['ImageID', 'XMin', 'XMax', 'YMin', 'YMax']], axis=1)

# Print the new dataframe
print(new_df.count())
new_df.head()


In [21]:
# Rename ImageID values to test/ + ImageID values
new_df['ImageID'] = new_df['ImageID'].astype(str) + '.jpg'
new_df['Family'] = 'Shark'
new_df['Genus'] = None
new_df['Species'] = None
new_df['Augmentation'] = None
new_df['Source'] = 'OpenImageV7'

# Rename ImageID column to filename
new_df.rename(columns={'ImageID': 'Filename'}, inplace=True)
new_df.rename(columns={'XMin': 'xmin'}, inplace=True)
new_df.rename(columns={'XMax': 'xmax'}, inplace=True)
new_df.rename(columns={'YMin': 'ymin'}, inplace=True)
new_df.rename(columns={'YMax': 'ymax'}, inplace=True)

# Reorder columns
new_df = new_df[['Filename', 'Family', 'Genus', 'Species', 'xmin', 'xmax', 'ymin', 'ymax', 'Augmentation', 'Source']]


In [19]:
new_df.head()

,ImageID,XMin,XMax,YMin,YMax
85227,008baba5cba24e0c,0.239167,0.776667,0.410625,0.762500
85228,008baba5cba24e0c,0.276667,0.662500,0.153125,0.398125
149756,014bc8fb008cf4e1,0.263246,0.868797,0.149471,0.898148
195502,01e4993063534510,0.145833,0.640000,0.248750,0.686875
208010,0211394ec63e3fd9,0.000000,0.238125,0.205441,0.422139


In [23]:
print(f'Total images: {human_verified_df.count()}, shark images: {shark_images.count()}, shark images with bounding boxes: {new_df.count()}')

Total images: ImageID       58783034
Source        58783034
LabelName     58783034
Confidence    58783034
dtype: int64, shark images: ImageID       1610
Source        1610
LabelName     1610
Confidence    1610
dtype: int64, shark images with bounding boxes: Filename        625
Family          625
Genus             0
Species           0
xmin            625
xmax            625
ymin            625
ymax            625
Augmentation      0
Source          625
dtype: int64


In [42]:
with open('openimagesv7_background_train.txt', 'a') as file:
  for image_id in set(bbox_filtered['ImageID']):
    file.write(f'train/{image_id}\n')


In [24]:
new_df.to_csv(f'../annotations/openimagesv7_{split}.csv', index=False)

# Clean Annotations
Remove annotations that contain files not present in the dataset (since I manually removed badly classified files .)

In [25]:
split = 'train'
annotation_df = pd.read_csv(f'../annotations/openimagesv7_{split}.csv')
annotation_df.head()

,Filename,Family,Genus,Species,xmin,xmax,ymin,ymax,Augmentation,Source
0,002b60dfc1de19ff.jpg,Shark,NaN,NaN,0.240625,0.89625,0.229823,0.702537,NaN,OpenImageV7
1,006e44adb308b8f7.jpg,Shark,NaN,NaN,0.000000,0.29000,0.834167,0.999167,NaN,OpenImageV7
2,006e44adb308b8f7.jpg,Shark,NaN,NaN,0.048750,0.25125,0.320833,0.475833,NaN,OpenImageV7
3,006e44adb308b8f7.jpg,Shark,NaN,NaN,0.148125,0.32625,0.555000,0.750000,NaN,OpenImageV7
4,006e44adb308b8f7.jpg,Shark,NaN,NaN,0.149375,0.33250,0.572500,0.761667,NaN,OpenImageV7


In [27]:
import os

files = os.listdir(f'../images/openimagesv7/{split}/')

In [29]:
# Filter the annotation_df dataframe based on the filename column in files
annotation_filtered = annotation_df[annotation_df['Filename'].isin(files)]

In [30]:
print(annotation_df.count())
print(annotation_filtered.count())

Filename        625
Family          625
Genus             0
Species           0
xmin            625
xmax            625
ymin            625
ymax            625
Augmentation      0
Source          625
dtype: int64
Filename        550
Family          550
Genus             0
Species           0
xmin            550
xmax            550
ymin            550
ymax            550
Augmentation      0
Source          550
dtype: int64


In [31]:
annotation_filtered.to_csv(f'../annotations/openimagesv7_{split}.csv', index=False)

In [36]:
# load train, val and test annotations and add .jpg to filename
for split in ['train', 'val', 'test']:
    annotation_df = pd.read_csv(f'../annotations/openimagesv7_{split}.csv')
    annotation_df['Filename'] = annotation_df['Filename'].apply(lambda x: x + '.jpg')
    annotation_df.to_csv(f'../annotations/openimagesv7_{split}.csv', index=False)